In [1]:
%load_ext autoreload
%autoreload 2

import gc

import os
import sys
import pandas as pd
import numpy as np
import tqdm
# import seaborn as sns

# pd.set_option('display.max_columns', None)

# если у вас есть CUDA, то она понадобится там для экспериментов в catboost
# os.environ["CUDA_VISIBLE_DEVICES"] = '0'

###Reading data

In [2]:
import zipfile
import os
import time

from google.colab import drive

In [3]:
import pandas as pd
import numpy as np

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
zip_file = '/content/drive/MyDrive/alfabattle2_test_transactions_contest.zip'

z = zipfile.ZipFile(zip_file, 'r')
z.extractall()

print(os.listdir())

FileNotFoundError: ignored

In [6]:
zip_file = '/content/drive/MyDrive/alfabattle2_train_transactions_contest.zip'

z = zipfile.ZipFile(zip_file, 'r')
z.extractall()

print(os.listdir())

FileNotFoundError: ignored

In [ ]:
zip_file = '/content/drive/MyDrive/alfabattle2.zip'

z = zipfile.ZipFile(zip_file, 'r')
z.extractall()

print(os.listdir())

['.config', 'alfabattle2_alpha_sample.csv', 'alfabattle2_train_target.csv', '__MACOSX', 'train_transactions_contest', 'alfabattle2_test_target_contest.csv', 'test_transactions_contest', 'drive', 'sample_data']


In [ ]:
zip_file = '/content/drive/MyDrive/utils_data.zip'

z = zipfile.ZipFile(zip_file, 'r')
z.extractall()

print(os.listdir())

['.config', 'alfabattle2_alpha_sample.csv', 'alfabattle2_train_target.csv', 'utils.py', 'batch_train_1.csv', '__MACOSX', 'train_transactions_contest', 'alfabattle2_test_target_contest.csv', 'test_transactions_contest', 'drive', 'sample_data']


In [ ]:
from utils import read_parquet_dataset_from_local

In [ ]:
TRAIN_TRANSACTIONS_PATH = '/content/train_transactions_contest'
TEST_TRANSACTIONS_PATH = '/content/test_transactions_contest'

TRAIN_TARGET_PATH = '/content/alfabattle2_train_target.csv'
TEST_TARGET_PATH = '/content/alfabattle2_test_target_contest.csv'

###Reading partitions

In [ ]:
def data_processing(transaction_path, targets_path, is_Test, num_parts):
  data = read_parquet_dataset_from_local(transaction_path, start_from=0, num_parts_to_read=num_parts)

  memory_usage_of_frame = data.memory_usage(index=True).sum() / 10**9
  expected_memory_usage = memory_usage_of_frame * 50
  print(f'Партиция : {0}')
  print(f'Объем памяти в  RAM одной партиции данных с транзакциями: {round(memory_usage_of_frame, 3)} Gb')
  print(f'Ожидаемый размер в RAM всего датасета: {round(expected_memory_usage, 3)} Gb')
  
  data.amnt.replace(0.0, np.nan, regex=False, inplace = True)
  targets = pd.read_csv(targets_path)
  if (is_Test):
    data = data.merge(targets[['app_id', 'product']], on='app_id')
  else:
    data = data.merge(targets[['app_id', 'product', 'flag']], on=['app_id'])
  
  data["Is_end_of_week"] = data['day_of_week'].isin([5,6,7])
  data["Is_holidays"] = data['weekofyear'].isin([1,2,9,10,18,19,51,52,53])

  data['amnt'] = data['amnt'].fillna(data.groupby('app_id')['amnt'].transform('mean'))
  return data, targets

In [ ]:
new_data, target_frame = data_processing(TRAIN_TRANSACTIONS_PATH, TRAIN_TARGET_PATH, False, 1)


Партиция : 0
Объем памяти в  RAM одной партиции данных с транзакциями: 0.476 Gb
Ожидаемый размер в RAM всего датасета: 23.798 Gb


In [ ]:
merged_test_data, test_target = data_processing(TEST_TRANSACTIONS_PATH, TEST_TARGET_PATH, True, 1)


Партиция : 0
Объем памяти в  RAM одной партиции данных с транзакциями: 0.305 Gb
Ожидаемый размер в RAM всего датасета: 15.236 Gb


In [ ]:
new_data

,app_id,amnt,currency,operation_kind,card_type,operation_type,operation_type_group,ecommerce_flag,payment_system,income_flag,mcc,country,city,mcc_category,day_of_week,hour,days_before,weekofyear,hour_diff,transaction_number,product,flag,Is_end_of_week,Is_holidays
0,0,0.465425,1,4,98,4,2,3,7,3,2,1,37,2,4,19,351,34,-1,1,3,0,False,False
1,0,0.390965,1,2,98,7,1,3,7,3,2,1,49,2,4,20,351,34,0,2,3,0,False,False
2,0,0.521152,1,2,98,3,1,3,7,3,2,1,37,2,4,20,351,34,0,3,3,0,False,False
3,0,0.356078,1,1,5,2,1,3,7,3,10,1,49,7,2,0,348,34,52,4,3,0,False,False
4,0,0.390965,1,2,98,7,1,3,7,3,2,1,49,2,4,16,337,53,280,5,3,0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5408643,23646,0.390944,1,1,37,2,1,1,3,1,1,1,3,1,6,9,2,48,32,453,1,0,True,False
5408644,23646,0.428447,1,1,37,2,1,1,3,1,39,1,120,7,5,12,1,48,27,454,1,0,True,False
5408645,23646,0.371478,1,1,37,2,1,1,3,1,22,1,2,9,5,13,1,48,1,455,1,0,True,False
5408646,23646,0.348726,1,1,37,2,1,1,3,1,22,1,3,9,5,13,1,48,1,456,1,0,True,False


In [ ]:
target_frame

,app_id,product,flag
0,0,3,0
1,1,1,0
2,2,1,0
3,3,1,0
4,4,1,0
...,...,...,...
963806,1003045,1,0
963807,1003047,0,0
963808,1003048,1,0
963809,1003049,0,0


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_data, val = train_test_split(target_frame, random_state=42, test_size=0.1)
train_data.shape, val.shape

((867429, 3), (96382, 3))

In [ ]:
train = new_data.loc[new_data['app_id'].isin(train_data.app_id.values)]

In [ ]:
train.shape

(4852424, 24)

In [ ]:
val_data = new_data.loc[new_data['app_id'].isin(val.app_id.values)]

In [ ]:
train.groupby("app_id").count()

,amnt,currency,operation_kind,card_type,operation_type,operation_type_group,ecommerce_flag,payment_system,income_flag,mcc,country,city,mcc_category,day_of_week,hour,days_before,weekofyear,hour_diff,transaction_number,product,flag,Is_end_of_week,Is_holidays
app_id,,,,,,,,,,,,,,,,,,,,,,,
0,181,181,181,181,181,181,181,181,181,181,181,181,181,181,181,181,181,181,181,181,181,181,181
1,356,356,356,356,356,356,356,356,356,356,356,356,356,356,356,356,356,356,356,356,356,356,356
2,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229,229
3,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67
4,117,117,117,117,117,117,117,117,117,117,117,117,117,117,117,117,117,117,117,117,117,117,117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23642,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161,161
23643,199,199,199,199,199,199,199,199,199,199,199,199,199,199,199,199,199,199,199,199,199,199,199
23644,404,404,404,404,404,404,404,404,404,404,404,404,404,404,404,404,404,404,404,404,404,404,404


###Datasets

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [ ]:
val_data.groupby("flag").count()

,app_id,amnt,currency,operation_kind,card_type,operation_type,operation_type_group,ecommerce_flag,payment_system,income_flag,mcc,country,city,mcc_category,day_of_week,hour,days_before,weekofyear,hour_diff,transaction_number,product,Is_end_of_week,Is_holidays
flag,,,,,,,,,,,,,,,,,,,,,,,
0,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219
1,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005


###Prepare the class for storing the information about the users' card spendings

In [ ]:
class TabularDataset(Dataset):
  def __init__(self, data, cat_cols=None, cont_cols=None, output_col=None, is_Train = True):
    """
    Characterizes a Dataset for PyTorch

    Parameters
    ----------

    data: pandas data frame
      The data frame object for the input data. It must
      contain all the continuous, categorical and the
      output columns to be used.

    cat_cols: List of strings
      The names of the categorical columns in the data.
      These columns will be passed through the embedding
      layers in the model. These columns must be
      label encoded beforehand. 

    output_col: string
      The name of the output variable column in the data
      provided.
    """
    self.is_Train = is_Train
    self.data = data.sort_values(['app_id', 'transaction_number'])

    self.unique_ids = data.app_id.unique()

    self.max_length = int(data[["app_id", "amnt"]].groupby(["app_id"]).count().max().values[0])
    self.max_id = int(data.iloc[data.shape[0]-1]["app_id"])

    self.seq_counts = [x[0] for x in (data[["app_id", "amnt"]].groupby(["app_id"]).count().values)]

    self.n = self.unique_ids.size#self.max_id + 1#data.shape[0]

    ## app_id of the dataframe
    #self.ids = merged_data["app_id"].astype(np.float32).values
    self.ids = {}
    k = 0
    for d in data["app_id"]:
      if (d not in self.ids.keys()):
        self.ids[d] = [k]
      else:
        self.ids[d].append(k)
      k += 1

    if output_col:
      self.y = data[output_col].astype(np.float32).values.reshape(-1, 1)#int64
    else:
      self.y =  np.zeros((data.shape[0], 1))

    self.cat_cols = cat_cols if cat_cols else []
    self.cont_cols = cont_cols  if cont_cols else []
    
    #[col for col in data.columns
    #                 if col not in self.cat_cols + [output_col]]

    if self.cont_cols:
      self.cont_X = data[self.cont_cols].astype(np.float32).values
    else:
      self.cont_X = np.zeros((self.n, 1))

    if self.cat_cols:
      self.cat_X = data[cat_cols].astype(np.int64).values
    else:
      self.cat_X =  np.zeros((self.n, 1))

  def __len__(self):
    """
    Denotes the total number of samples.
    """
    return self.n

  def __getitem__(self, id):
    """
    Generates one sample of data.
    idx is a number of app_id
    """
    #print(self.unique_ids)
    idx = self.unique_ids[id]
    #print(self.unique_ids[id])
    
    #target = np.zeros(2, dtype=np.int64)#len(self.ids[idx])
    target = np.zeros(len(self.ids[idx]), dtype=np.float32)#int64
    out_cat = np.zeros((len(self.ids[idx]), len(self.cat_cols)), dtype=np.int64) # self.max_length # len(self.cat_cols + self.cont_cols)
    out_cont = np.zeros((len(self.ids[idx]), len(self.cont_cols)), dtype=np.float32) # self.max_length
    #print(out_cat.shape)
    k = 0
    for i in self.ids[idx]:
      #print(self.ids[idx])
      target[k] = self.y[i]
      #print(self.cont_X[i])
      out_cont[k] = self.cont_X[i]#, dtype=np.float32)
      out_cat[k] = self.cat_X[i]
      k += 1
    #target[int(np.mean(self.y[i]))] = 1
    tar = np.mean(target)
      #out.append([self.y[i], self.cont_X[i], self.cat_X[i]])
    if (self.is_Train):
      return (tar, out_cat, out_cont, self.seq_counts[id])
    else:
      return (idx, out_cat, out_cont, self.seq_counts[id])

In [ ]:
cat_vars = ['currency', 'operation_kind', 'card_type',
               'operation_type', 'operation_type_group', 'ecommerce_flag',
               'payment_system', 'income_flag', 'mcc', 'country', 'city',
               'mcc_category', 'day_of_week', 'hour','weekofyear', 'Is_end_of_week', 'Is_holidays', 'product']

cont_vars = ['amnt', 'days_before', 'hour_diff']
dep_var = 'flag'

In [ ]:
from sklearn import preprocessing

In [ ]:
import torch
import numpy as np

class PadSequence:
    def __call__(self, batch):
		  # Let's assume that each element in "batch" is a tuple (data, label).
      # Sort the batch in the descending order
      #print(batch)
      sorted_batch = sorted(batch, key=lambda x: x[3], reverse=True)
		  # Get each sequence and pad it
      
      cat = [torch.LongTensor(x[1]) for x in sorted_batch]
      #print(cat[2].shape)
      cat_padded = torch.nn.utils.rnn.pad_sequence(cat, batch_first=True)
      #print(cat_padded[2].shape)
      cont = [torch.FloatTensor(preprocessing.StandardScaler().fit(x[2]).transform(x[2])) for x in sorted_batch]
      cont_padded = torch.nn.utils.rnn.pad_sequence(cont, batch_first=True)
		  # Also need to store the length of each sequence
		  # This is later needed in order to unpad the sequences
      lengths = torch.LongTensor([x[3] for x in sorted_batch])
      # Don't forget to grab the labels of the *sorted* batch
      labels = torch.FloatTensor([x[0] for x in sorted_batch])
      return labels, cat_padded, cont_padded, lengths

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoders = {}
for cat_col in cat_vars:
  label_encoders[cat_col] = LabelEncoder()
  #train[cat_col] = label_encoders[cat_col].fit_transform(train[cat_col])
  #val_data[cat_col] = label_encoders[cat_col].fit_transform(val_data[cat_col])
  merged_test_data[cat_col] = label_encoders[cat_col].fit_transform(merged_test_data[cat_col])

In [ ]:
dataset = TabularDataset(data=train, cat_cols=cat_vars, cont_cols = cont_vars, output_col=dep_var)

In [ ]:
dataset_val = TabularDataset(data=val_data, cat_cols=cat_vars, cont_cols = cont_vars, output_col=dep_var)

In [ ]:
dataset_test = TabularDataset(data=merged_test_data, cat_cols=cat_vars, cont_cols = cont_vars, is_Train = False)

In [ ]:
len(dataset_test)

10522

In [ ]:
dataset_test[0]

(1063620, array([[0, 0, 4, ..., 0, 0, 0],
        [0, 0, 4, ..., 0, 0, 0],
        [0, 0, 4, ..., 0, 0, 0],
        ...,
        [0, 0, 4, ..., 1, 0, 0],
        [0, 0, 4, ..., 1, 0, 0],
        [0, 0, 4, ..., 0, 0, 0]]), array([[ 3.6768672e-01,  3.5900000e+02, -1.0000000e+00],
        [ 4.0360665e-01,  3.5800000e+02,  1.9000000e+01],
        [ 3.5666594e-01,  3.5800000e+02,  0.0000000e+00],
        ...,
        [ 4.0348879e-01,  5.0000000e+00,  4.0000000e+00],
        [ 3.5578224e-01,  5.0000000e+00,  1.0000000e+00],
        [ 5.3325504e-01,  1.0000000e+00,  8.9000000e+01]], dtype=float32), 673)

In [ ]:
merged_test_data.groupby("app_id").count()

,amnt,currency,operation_kind,card_type,operation_type,operation_type_group,ecommerce_flag,payment_system,income_flag,mcc,country,city,mcc_category,day_of_week,hour,days_before,weekofyear,hour_diff,transaction_number,product,Is_end_of_week,Is_holidays
app_id,,,,,,,,,,,,,,,,,,,,,,
1063620,673,673,673,673,673,673,673,673,673,673,673,673,673,673,673,673,673,673,673,673,673,673
1063621,219,219,219,219,219,219,219,219,219,219,219,219,219,219,219,219,219,219,219,219,219,219
1063622,895,895,895,895,895,895,895,895,895,895,895,895,895,895,895,895,895,895,895,895,895,895
1063623,630,630,630,630,630,630,630,630,630,630,630,630,630,630,630,630,630,630,630,630,630,630
1063624,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127,127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1074458,290,290,290,290,290,290,290,290,290,290,290,290,290,290,290,290,290,290,290,290,290,290
1074459,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22
1074460,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265


In [ ]:
dataset_test[0]

(1063620, array([[0, 0, 4, ..., 0, 0, 0],
        [0, 0, 4, ..., 0, 0, 0],
        [0, 0, 4, ..., 0, 0, 0],
        ...,
        [0, 0, 4, ..., 1, 0, 0],
        [0, 0, 4, ..., 1, 0, 0],
        [0, 0, 4, ..., 0, 0, 0]]), array([[ 3.6768672e-01,  3.5900000e+02, -1.0000000e+00],
        [ 4.0360665e-01,  3.5800000e+02,  1.9000000e+01],
        [ 3.5666594e-01,  3.5800000e+02,  0.0000000e+00],
        ...,
        [ 4.0348879e-01,  5.0000000e+00,  4.0000000e+00],
        [ 3.5578224e-01,  5.0000000e+00,  1.0000000e+00],
        [ 5.3325504e-01,  1.0000000e+00,  8.9000000e+01]], dtype=float32), 673)

In [ ]:
val_data.groupby("flag").count()

,app_id,amnt,currency,operation_kind,card_type,operation_type,operation_type_group,ecommerce_flag,payment_system,income_flag,mcc,country,city,mcc_category,day_of_week,hour,days_before,weekofyear,hour_diff,transaction_number,product,Is_end_of_week,Is_holidays
flag,,,,,,,,,,,,,,,,,,,,,,,
0,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219,545219
1,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005,11005


###Weighted Random Sampler

In [ ]:
target_list = torch.tensor([i[0] for i in dataset], dtype = torch.int64)

In [ ]:
target_list.size()

torch.Size([20445])

In [ ]:
def get_class_distribution(dataset_obj):
    count_dict = {0:0, 1:0}
    
    for element in dataset_obj:
        #print(element[1])
        count_dict[element[0]] += 1
            
    return count_dict
print("Distribution of classes: \n", get_class_distribution(dataset))

Distribution of classes: 
 {0: 19860, 1: 585}


In [ ]:
class_count = [i for i in get_class_distribution(dataset).values()]

In [ ]:
class_weights = 1./torch.tensor(class_count, dtype=torch.float) 
class_weights

tensor([5.0352e-05, 1.7094e-03])

In [ ]:
class_weights_all = class_weights[target_list]
print(class_weights_all)
print(len(class_weights_all))

tensor([5.0352e-05, 5.0352e-05, 5.0352e-05,  ..., 5.0352e-05, 5.0352e-05,
        5.0352e-05])
20445


In [ ]:
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler

In [ ]:
weighted_sampler = WeightedRandomSampler(
    weights=class_weights_all,
    num_samples=len(class_weights_all),
    replacement=True
)

Validation

In [ ]:
target_list_val = torch.tensor([i[0] for i in dataset_val], dtype = torch.int64)

In [ ]:
target_list_val.size()

torch.Size([2298])

In [ ]:
print("Distribution of classes for validation: \n", get_class_distribution(dataset_val))

Distribution of classes for validation: 
 {0: 2238, 1: 60}


In [ ]:
class_count_val = [i for i in get_class_distribution(dataset_val).values()]
class_count_val

[2238, 60]

In [ ]:
class_weights_val = 1./torch.tensor(class_count_val, dtype=torch.float)
class_weights_val

tensor([0.0004, 0.0167])

In [ ]:
class_weights_all_val = class_weights_val[target_list_val]
print(class_weights_all_val)
print(len(class_weights_all_val))

tensor([0.0004, 0.0004, 0.0004,  ..., 0.0004, 0.0004, 0.0004])
2298


In [ ]:
len(class_weights_all_val)

2298

In [ ]:
weighted_sampler_val = WeightedRandomSampler(
    weights=class_weights_all_val,
    num_samples=len(class_weights_all_val),
    replacement=True
)

###Dataloader

In [ ]:
batchsize = 128
dataloader = DataLoader(dataset=dataset, shuffle=False, batch_size=batchsize, sampler=weighted_sampler, collate_fn=PadSequence())

###Model

In [ ]:
class Model(nn.Module):
    def __init__(self, emb_dims, no_of_cont, emb_dropout, lin_dropout, top_classifier_units, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Embedding layers
        self.emb_layers = nn.ModuleList([nn.Embedding(x, y)
                                     for x, y in emb_dims])

        no_of_embs = sum([y for x, y in emb_dims])
        self.no_of_embs = no_of_embs
        self.no_of_cont = no_of_cont

        # Batch Norm Layers
        #self.first_bn_layer = nn.BatchNorm1d(3)#self.no_of_cont)
        #self.bn_layers = nn.ModuleList([nn.BatchNorm1d(size)
        #                            for size in lin_layer_sizes])
        
        # Dropout Layers
        self.emb_dropout_layer = nn.Dropout(emb_dropout)
        #self.droput_layers = nn.ModuleList([nn.Dropout(size)
        #                          for size in lin_layer_dropouts])
        
        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        #self.rnn = nn.RNN(self.no_of_embs + self.no_of_cont, hidden_dim, n_layers, batch_first=True)   
        
        self.lstm = nn.GRU(self.no_of_embs + self.no_of_cont, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=False,
                           batch_first=True)
        
        self.top_classifier = nn.Linear(hidden_dim, top_classifier_units)
        self.intermediate_activation = nn.ReLU()

        # Fully connected layer
        self.fc = nn.Linear(top_classifier_units, output_size)

        self.dropout = nn.Dropout(lin_dropout)

        #activation function
        self.act = nn.Sigmoid()
        #self.act = nn.Softmax(dim = 1)#
    
    def forward(self, cont_data, cat_data, seq_lengths):

##embedding layers
        if self.no_of_embs != 0:
          x = [emb_layer(cat_data[:,:,i])
            for i,emb_layer in enumerate(self.emb_layers)]
          #print(cont_data.shape)
          #print("Shape ", len(x), x[1].shape)
          x = torch.cat(x, 2)
          x = self.emb_dropout_layer(x)

        #print(cont_data.shape)
        #if self.no_of_cont != 0:
          #print(cont_data.shape)
          #print(cont_data[:].shape)
          #normalized_cont_data = self.first_bn_layer(cont_data[:])

        normalized_cont_data = cont_data

        if self.no_of_embs != 0:
          x = torch.cat([x, normalized_cont_data], 2) 
        else:
          x = normalized_cont_data
        ## emb

        #packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(x, seq_lengths.cpu(),batch_first=True, enforce_sorted=False)
        #print("packed_emb", packed_embedded)
        #print("packed_emb", packed_embedded.data[0].shape)

        batch_size = x.size(0)
        #print("Batch_size ", batch_size)

        #Initializing hidden state for first input using method defined below
        #hidden = self.init_hidden(batch_size)

        ###LSTM
        #packed_output, (hidden, cell) = self.lstm(packed_embedded)
        _, hidden = self.lstm(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
        
        #concat the final forward and backward hidden state
        #hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
        #hidden = [batch size, hid dim * num directions]
        classification_hidden = self.top_classifier(self.dropout(hidden))
        activation = self.intermediate_activation(classification_hidden)

        dense_outputs=self.fc(activation)

        #Final activation function
        outputs=self.act(dense_outputs)

        # Passing in the input and hidden state into the model and obtaining outputs
        #out, hidden = self.rnn(packed_embedded, hidden)

        #row_indices = torch.arange(0, x.size(0)).long()
        #col_indices = torch.LongTensor([355])
        #if next(self.parameters()).is_cuda:
        #    row_indices = row_indices.cuda()
        #    col_indices = col_indices.cuda()

        #if self.use_last:
        #last_tensor=out[row_indices, col_indices, :]
        #else:
        #    # use mean
        #    last_tensor = out[row_indices, :, :]
        #    last_tensor = torch.mean(last_tensor, dim=1)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        #out = out.contiguous().view(-1, self.hidden_dim)
        #print("RNN ", last_tensor)
        #out = self.fc(out)
        
        return outputs
    
    #def init_hidden(self, batch_size):
    #    # This method generates the first hidden state of zeros which we'll use in the forward pass
    #    hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
    #     # We'll send the tensor holding the hidden state to the device we specified earlier as well
    #    return hidden

In [ ]:
cat_dims = [int(new_data[col].nunique()) for col in cat_vars]
print(cat_dims)

emb_dims = [(x, min(50, (x + 1) // 2)) for x in cat_dims]
emb_dims

[11, 7, 121, 20, 3, 3, 6, 3, 108, 24, 161, 28, 7, 24, 53, 2, 2, 4]


[(11, 6),
 (7, 4),
 (121, 50),
 (20, 10),
 (3, 2),
 (3, 2),
 (6, 3),
 (3, 2),
 (108, 50),
 (24, 12),
 (161, 50),
 (28, 14),
 (7, 4),
 (24, 12),
 (53, 27),
 (2, 1),
 (2, 1),
 (4, 2)]

In [ ]:
emb_dims = [(11, 6), 
            (7, 5),
            (175, 29),
            (22, 9),
            (4, 3),
            (3, 3),
            (7, 5),
            (3, 3),
            (108, 22),
            (24, 9),
            (163, 28),
            (28, 10),
            (7, 5),
            (24, 9),
            (53, 15),
            (2, 1),
            (2, 1),
            (4, 2)]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model(emb_dims, no_of_cont=3, emb_dropout=0.04, lin_dropout = 0.5, top_classifier_units = 32, output_size=1, hidden_dim=128, n_layers=1).to(device)

In [ ]:
model

Model(
  (emb_layers): ModuleList(
    (0): Embedding(11, 6)
    (1): Embedding(7, 4)
    (2): Embedding(121, 50)
    (3): Embedding(20, 10)
    (4): Embedding(3, 2)
    (5): Embedding(3, 2)
    (6): Embedding(6, 3)
    (7): Embedding(3, 2)
    (8): Embedding(108, 50)
    (9): Embedding(24, 12)
    (10): Embedding(161, 50)
    (11): Embedding(28, 14)
    (12): Embedding(7, 4)
    (13): Embedding(24, 12)
    (14): Embedding(53, 27)
    (15): Embedding(2, 1)
    (16): Embedding(2, 1)
    (17): Embedding(4, 2)
  )
  (emb_dropout_layer): Dropout(p=0.04, inplace=False)
  (lstm): GRU(255, 128, batch_first=True)
  (top_classifier): Linear(in_features=128, out_features=32, bias=True)
  (intermediate_activation): ReLU()
  (fc): Linear(in_features=32, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (act): Sigmoid()
)

In [ ]:
import torch.optim as optim

#define optimizer and loss
optimizer = optim.Adam(model.parameters())
#criterion = nn.BCELoss()

#define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(preds)
    #rounded_preds = torch.argmax(preds, dim = 1)
    #print(rounded_preds)

    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
#push to cuda if available
model = model.to(device)


In [ ]:
from sklearn.metrics import roc_auc_score

def roc_auc_score_s(preds, y) -> float:
    """
    функция для оценки качества модели на отложенной выборке, возвращает roc-auc на валидационной
    выборке
    :param model: nn.Module модель
    :param dataset_val: путь до директории с последовательностями
    :param batch_size: размер батча
    :param device: device, на который будут положены данные внутри батча
    :return: val roc-auc score
    """
    output = []
    targets = []
    #print("y ", y)
    #print("pr ", preds)
    targets.extend(y.detach().cpu().numpy().flatten())
    #output = model(batch['transactions_features'], batch['product'])
    output.extend(preds.detach().cpu().numpy().flatten())
    #rounded_preds = torch.round(preds)
    #print(output)

    return roc_auc_score(targets, output)

In [ ]:
def eval_model(model, dataset_val, weighted_sampler) -> float:
    """
    функция для оценки качества модели на отложенной выборке, возвращает roc-auc на валидационной
    выборке
    :param model: nn.Module модель
    :param dataset_val: путь до директории с последовательностями
    :param batch_size: размер батча
    :param device: device, на который будут положены данные внутри батча
    :return: val roc-auc score
    """
    preds = []
    targets = []
    batchsize = 64
    val_dataloader = DataLoader(dataset=dataset_val, shuffle=False, batch_size=batchsize, sampler=weighted_sampler, collate_fn=PadSequence())
    model.eval()
    

    for y, cat_x, cont_x, seq_length in val_dataloader:
      cat_x = cat_x.to(device)
      cont_x = cont_x.to(device)
      y  = y.to(device)

      preds = model(cont_x, cat_x, seq_length).squeeze()

      acc = roc_auc_score_s(preds, y)

    return acc.item()

In [ ]:
def inference(model, dataset_test, batch_size=32) -> pd.DataFrame:
    """
    функция, которая делает предикты на новых данных, возвращает pd.DataFrame из двух колонок:
    (app_id, score)
    :param model: nn.Module модель
    :param dataset_test: путь до директории с последовательностями
    :param batch_size: размер батча
    :param device: device, на который будут положены данные внутри батча
    :return: pd.DataFrame из двух колонок: (app_id, score)
    """
    model.eval()
    preds = []
    app_ids = []
    test_dataloader = DataLoader(dataset=dataset_test, shuffle=False, batch_size=batch_size, collate_fn=PadSequence())

    for app_id, cat_x, cont_x, seq_length in test_dataloader:
        cat_x = cat_x.to(device)
        cont_x = cont_x.to(device)
        
        app_ids.extend(app_id)
        output =  model(cont_x, cat_x, seq_length).squeeze()
        preds.extend(output.detach().cpu().numpy().flatten())
        
    return pd.DataFrame({
        'app_id': app_ids,
        'score': preds
    })

In [ ]:
test_dataloader = DataLoader(dataset=dataset_test, shuffle=False, batch_size=64, collate_fn=PadSequence())

In [ ]:
dat = next(iter(test_dataloader))
  
print(len(dat))
print(dat[2][0])
print(dat[2][0].mean(0))
print(dat[2][0].var(0))


4
tensor([[ 1.8033,  1.9097, -0.6562],
        [ 1.7749,  1.9097, -0.5535],
        [-0.9464,  1.8998,  0.2682],
        ...,
        [-3.7010, -1.6255, -0.5535],
        [ 0.4854, -1.6255, -0.1427],
        [ 0.2447, -1.6255, -0.5535]])
tensor([ 9.6209e-09, -3.3673e-08, -6.3137e-09])
tensor([1.0006, 1.0006, 1.0006])


In [ ]:
def train_epoch(model, optimizer, dataset_train, weighted_sampler, batch_size=64, print_loss_every_n_batches=500):
    """
    делает одну эпоху обучения модели, логирует
    :param model: nn.Module модель
    :param optimizer: nn.optim оптимизатор
    :param dataset_train: путь до директории с последовательностями
    :param batch_size: размерм батча
    :param shuffle: флаг, если True, то перемешивает данные
    :param print_loss_every_n_batches: число батчей после которых логируется лосс на этих батчах
    :param device: device, на который будут положены данные внутри батча
    :return: None
    """
    dataloader = DataLoader(dataset=dataset_train, shuffle=False, batch_size=batch_size, sampler=weighted_sampler, collate_fn=PadSequence())

    criterion = nn.BCELoss()
    criterion = criterion.to(device)

    num_batches = 1
    running_loss = 0.0
    epoch_acc = 0.0

    model.train()

    for y, cat_x, cont_x, seq_length in dataloader:
        cat_x = cat_x.to(device)
        cont_x = cont_x.to(device)
        y  = y.to(device)
        seq_length = seq_length.to(device)

        preds = model(cont_x, cat_x, seq_length).squeeze()

        loss = criterion(preds, y)

        acc = roc_auc_score_s(preds, y) 

        optimizer.zero_grad()
        #print(loss)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()  
        epoch_acc += acc.item()  

        if num_batches % print_loss_every_n_batches == 0:
            print(f'Training loss after {num_batches} batches: {running_loss / num_batches} roc-auc: {epoch_acc / num_batches}', end='\r')
        
        num_batches += 1
    
    print(f'Training loss after epoch: {running_loss / num_batches}', end='\r')  

In [ ]:
import torch
import numpy as np


class EarlyStopping:
    """Early stops the training if validation metric doesn't improve after a given patience."""

    def __init__(self, patience=7, mode='min', verbose=False, delta=0, save_path='checkpoint.hdf5', metric_name=None, save_format='torch'):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            mode (str): One of ['min', 'max'], whether to maximize or minimaze the metric.
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            save_path (str): Path to saved model
        """
        if mode not in ['min', 'max']:
            raise ValueError(f'Unrecognized mode: {mode}!')

        self.patience = patience
        self.mode = mode
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.best_prev_score = np.Inf if mode == 'min' else -np.Inf
        self.delta = delta
        self.save_path = save_path
        self.metric_name = 'metric' if not metric_name else metric_name
        if save_format not in ['torch', 'tf']:
            raise ValueError('Expected to save in one of the following formats: ["torch", "tf"]')
        self.save_format = save_format
        
    def __call__(self, metric_value, model):

        score = -metric_value if self.mode == 'min' else metric_value

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(metric_value, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(
                f'No imporvement in Validation {self.metric_name}. Current: {score:.6f}. Current best: {self.best_score:.6f}')
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(metric_value, model)
            self.counter = 0

    def save_checkpoint(self, metric_value, model):
        """Saves model when validation loss decrease."""
        if self.verbose:
            print(
                f'Validation {self.metric_name} improved ({self.best_prev_score:.6f} --> {metric_value:.6f}).  Saving model ...')
        if self.save_format == 'tf':
            model.save_weights(self.save_path)
        else:
            torch.save(model.state_dict(), self.save_path)
            print ('model saved in ', self.save_path)
            
        self.best_prev_score = metric_value


In [ ]:
!ls /content/rnn_baseline/checkpoints

In [ ]:
!mkdir -p /content/rnn_baseline/checkpoints

In [ ]:
#! rm -r /content/rnn_baseline/checkpoints
#! mkdir -p /content/rnn_baseline/checkpoints

In [ ]:
path_to_checkpoints = '/content/rnn_baseline/checkpoints/'
es = EarlyStopping(patience=3, mode='max', verbose=True, save_path=os.path.join(path_to_checkpoints, 'best_checkpoint.pt'), 
                   metric_name='ROC-AUC', save_format='torch')

In [ ]:
optimizer = torch.optim.Adam(lr=1e-3, params=model.parameters())
num_epochs = 15
train_batch_size = 128
val_batch_szie = 128

In [ ]:
for epoch in range(num_epochs):
    print(f'Starting epoch {epoch+1}')
    train_epoch(model, optimizer, dataset, weighted_sampler, batch_size=128, print_loss_every_n_batches=500)
    
    val_roc_auc = eval_model(model, dataset_val, weighted_sampler_val)
    es(val_roc_auc, model)
    
    if es.early_stop:
        print('Early stopping reached. Stop training...')
        break
    torch.save(model.state_dict(), os.path.join(path_to_checkpoints, f'epoch_{epoch+1}_val_{val_roc_auc:.3f}.pt'))
    
    train_roc_auc = eval_model(model, dataset, weighted_sampler)
    print(f'Epoch {epoch+1} completed. Train roc-auc: {train_roc_auc}, Val roc-auc: {val_roc_auc}')

Starting epoch 1


IndexError: ignored

In [ ]:
criterion = nn.BCELoss()
criterion = criterion.to(device)

In [ ]:
no_of_epochs = 5
#criterion = nn.MSELoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

epoch_loss = 0
epoch_acc = 0
batch_size = 128

for epoch in range(no_of_epochs):
  model.train()
  epoch_loss = 0
  epoch_acc = 0
  n = 0
  for y, cat_x, cont_x, seq_length in dataloader:
    cat_x = cat_x.to(device)
    cont_x = cont_x.to(device)
    y  = y.to(device)
    seq_length = seq_length.to(device)
    #print(cat_x)
    #print(cont_x)
    #print(y)
    #print(seq_length)

    # Forward Pass
    preds = model(cont_x, cat_x, seq_length).squeeze()
    #print("Preds ", preds[0].shape)
    
    #print(y)
    loss = criterion(preds, y)
    #print("Preds ", preds)
    #compute the binary accuracy
    #acc = binary_accuracy(preds, y)
    acc = roc_auc_score_s(preds, y)   
    
    # Backward Pass and Optimization
    optimizer.zero_grad()
    #print(loss)
    loss.backward()
    optimizer.step()

    #loss and accuracy
    epoch_loss += loss.item()  
    epoch_acc += acc.item()  
    n += 1
  print("n", n)
  print(f'\tTrain Loss: {epoch_loss / n:.3f} | Train Acc: {epoch_acc / n:.2f}%')
  #print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

  val_roc_auc = eval_model(model, dataset_val, weighted_sampler_val)
  print(f'Epoch {epoch+1} completed. Val roc-auc: {val_roc_auc}')

IndexError: ignored

###Test Dataset

In [ ]:
! ls $path_to_checkpoints

In [ ]:
model.load_state_dict(torch.load(os.path.join(path_to_checkpoints, 'best_checkpoint.pt')))

<All keys matched successfully>

In [ ]:
test_preds = inference(model, dataset_test, batch_size=128)

IndexError: ignored

In [ ]:
test_preds.head()

In [ ]:
test_preds.to_csv('rnn_baseline_submission.csv', index=None) # ~ 0.750 на public test